In [9]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.ensemble import IsolationForest
import json

# This tells pandas to show all columns
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

Libraries imported successfully!


In [10]:
print("\nStarting Data Ingestion")
log_dir = './cloudtrail_logs/CloudTrail_dataset/'

file_path = os.path.join(log_dir , '**/*.json')
log_files = glob.glob(file_path , recursive = True)

print(f"Found {len(log_files)} log files.")
allrecords = []

for file in log_files:
    try:
        with open(file,'r') as f:
            data = json.load(f)

            if 'Records' in data :
                allrecords.extend(data['Records'])
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file: {file}")
    except Exception as e:
        print(f"An error occurred with file {file}: {e}")
        
print(f"\nSuccessfully loaded a total of {len(allrecords)} log events.")
        

Found 55 log files.

Successfully loaded a total of 2900 log events.


In [14]:
print("\nStarting flattening Json into a dataframe ")

try:
    df = pd.json_normalize(allrecords)
    print(f"Successfully normalized {len(allrecords)} log events")

    print(f"Dataframe shape{df.shape} rows ,columns")

    print("\n--Dataframe column list (df.info())--")
    df.info()


except Exception as e:
    print(f"An error occurred during normalization: {e}")
      
      

Starting flattening Json into a dataframe 
Successfully normalized 2900 log events
Dataframe shape(2900, 694) rows ,columns

--Dataframe column list (df.info())--
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Columns: 694 entries, eventVersion to requestParameters.lookupAttributes
dtypes: bool(2), float64(71), object(621)
memory usage: 15.3+ MB


In [15]:
print("\nStarting Cleaning and selecting features ")

try:
    df['eventTime'] = pd.to_datetime(df['eventTime'])
except Exception as e :
    print(f"could not convert eventTime :{e}")

    
core_columns = [
    'eventTime',
    'eventName',
    'eventSource',
    'awsRegion',
    'sourceIPAddress',
    'userAgent',
    'errorCode',  # This will be 'NaN' (Not a Number) for successful events
    
    # We need to find the best column to identify the 'actor' or 'user'.
    # It's usually one of these. We'll check which one is best.
    'userIdentity.type',
    'userIdentity.principalId',
    'userIdentity.arn',
    'userIdentity.userName'
]

columns_to_keep = [col for col in core_columns if col in df.columns]

df_clean = df[columns_to_keep].copy()

print("\n--- Clean DataFrame (df_clean.head()) ---")
print(df.head())

print("\n--- Null values in our new DataFrame ---")
# This shows us how many 'NaN' (empty) values are in each column
print(df_clean.isnull().sum())



Starting Cleaning and selecting features 

--- Clean DataFrame (df_clean.head()) ---
  eventVersion                 eventTime       eventSource  \
0         1.08 2023-07-10 11:42:36+00:00  s3.amazonaws.com   
1         1.08 2023-07-10 11:42:44+00:00  s3.amazonaws.com   
2         1.08 2023-07-10 11:42:44+00:00  s3.amazonaws.com   
3         1.08 2023-07-10 11:42:44+00:00  s3.amazonaws.com   
4         1.08 2023-07-10 11:42:44+00:00  s3.amazonaws.com   

                     eventName  awsRegion sourceIPAddress  \
0  GetStorageLensConfiguration  us-east-1    AWS Internal   
1   GetBucketPublicAccessBlock  us-east-1    10.248.16.43   
2        GetBucketPolicyStatus  us-east-1    10.248.16.43   
3                 GetBucketAcl  us-east-1    10.248.16.43   
4   GetBucketPublicAccessBlock  us-east-1    10.248.16.43   

                                           userAgent  responseElements  \
0                                       AWS Internal               NaN   
1  [S3Console/0.4, aws-int

In [16]:
print("Starting  Feature Engineering...")

# Define the column we will use to group 'actors'
ENTITY_ID = 'userIdentity.principalId'

# Drop rows where this ID is missing, as we can't profile them
df_clean = df_clean.dropna(subset=[ENTITY_ID])
print(f"Working with {len(df_clean)} events that have a valid '{ENTITY_ID}'.")


# --- 1. Engineer helper columns ---

# Create 'is_error' column: 1 if errorCode is NOT null, 0 if it is null
df_clean['is_error'] = df_clean['errorCode'].notnull().astype(int)

# Create 'is_night_activity' column: 1 if event was between 12 AM and 5 AM
df_clean['is_night_activity'] = (df_clean['eventTime'].dt.hour >= 0) & (df_clean['eventTime'].dt.hour <= 5)
df_clean['is_night_activity'] = df_clean['is_night_activity'].astype(int)

print("Engineered helper columns 'is_error' and 'is_night_activity'.")


# --- 2. Group by entity and aggregate behaviors ---

print(f"\nGrouping by '{ENTITY_ID}' to build behavioral profiles...")

df_behavior = df_clean.groupby(ENTITY_ID).agg(
    # Total number of API calls
    event_count=pd.NamedAgg(column='eventName', aggfunc='count'),
    
    # Total number of errors (e.g., AccessDenied)
    error_count=pd.NamedAgg(column='is_error', aggfunc='sum'),
    
    # How many different services they touched (e.g., S3, EC2, IAM)
    unique_services_used=pd.NamedAgg(column='eventSource', aggfunc='nunique'),
    
    # How many different regions they operated in
    unique_regions=pd.NamedAgg(column='awsRegion', aggfunc='nunique'),
    
    # How many different IP addresses they used
    unique_ips=pd.NamedAgg(column='sourceIPAddress', aggfunc='nunique'),
    
    # How many of their actions were at night
    night_events=pd.NamedAgg(column='is_night_activity', aggfunc='sum')
)

print("Successfully created behavioral DataFrame.")


# --- 3. Inspect the new DataFrame ---
print("\n--- Behavioral Profiles (df_behavior.head()) ---")
print(df_behavior.head())

print(f"\nNew DataFrame shape: {df_behavior.shape}")
print("This table is now ready for the model.")

Starting  Feature Engineering...
Working with 2824 events that have a valid 'userIdentity.principalId'.
Engineered helper columns 'is_error' and 'is_night_activity'.

Grouping by 'userIdentity.principalId' to build behavioral profiles...
Successfully created behavioral DataFrame.

--- Behavioral Profiles (df_behavior.head()) ---
                                                    event_count  error_count  \
userIdentity.principalId                                                       
AIDATFQR7NSC5AU2ZV3IE                                      2642          239   
AIDATFQR7NSC5U6Q3TMDR                                       105           14   
AIDATFQR7NSCYG26CT6RI                                         1            0   
AROATFQR7NSC3K2SEQDM2:MandoService2842426183934...            1            0   
AROATFQR7NSC3K2SEQDM2:MandoService3640611795397...            1            0   

                                                    unique_services_used  \
userIdentity.principalId        

In [17]:
print("Starting  Anomaly Detection...")

# --- 1. Initialize the model ---

# contamination='auto' is a good default, but for small datasets
# like this, setting it manually is more reliable.
# Let's assume ~10% of our entities might be anomalous.
# This is a hyperparameter you can tune.
model = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)

# --- 2. Fit the model to our data ---
# The model learns the "shape" of normal behavior from our 13 profiles
model.fit(df_behavior)

# --- 3. Get the predictions ---
# .predict() gives 1 for 'inlier' (normal) and -1 for 'outlier' (anomaly)
predictions = model.predict(df_behavior)

# We can also get the raw anomaly "score"
# Lower scores are more anomalous.
scores = model.decision_function(df_behavior)

# --- 4. Add predictions back to our DataFrame ---
df_behavior['anomaly_score'] = scores
df_behavior['is_anomaly'] = predictions

print("Model fitting and prediction complete.")

# --- 5. Inspect the results ---
print("\n--- DataFrame with Anomaly Predictions ---")
# Sort by the score to see the most anomalous entities at the top
df_behavior_results = df_behavior.sort_values(by='anomaly_score', ascending=True)

print(df_behavior_results)

Starting  Anomaly Detection...
Model fitting and prediction complete.

--- DataFrame with Anomaly Predictions ---
                                                    event_count  error_count  \
userIdentity.principalId                                                       
AIDATFQR7NSC5AU2ZV3IE                                      2642          239   
AIDATFQR7NSC5U6Q3TMDR                                       105           14   
AROATFQR7NSC6Q6YRQ2Q7:i-0dbc91f429e48eeed                    15            0   
AROATFQR7NSCWWVLB7BES:aws-go-sdk-16889900825233...           29           29   
AROATFQR7NSCWCZMFXMXZ:aws-go-sdk-16889905652861...           15           15   
AROATFQR7NSCQNEXZHIOB:i-05c30218156bcc246                     8            0   
AROATFQR7NSCRR66DMFTC:SLRManagement                           4            0   
AROATFQR7NSC4RHD6IN2N:aws-go-sdk-16889907971034...            1            1   
AROATFQR7NSCYGNOX4QAJ:aws-go-sdk-16889909660846...            1            1   
AROATF

In [18]:
print("Starting  Reporting Anomalies...")

# Filter the DataFrame to get only the anomalies
anomalies = df_behavior_results[df_behavior_results['is_anomaly'] == -1]

if anomalies.empty:
    print("\n--- ✅ No Anomalies Detected ---")
    print("The model did not flag any entities as anomalous.")
    
else:
    print(f"\n--- 🚨 {len(anomalies)} Anomalous Entities Detected! ---")
    
    # Loop through each anomalous row (each flagged actor)
    # .iterrows() lets us access the index (the principalId) and the row data
    for principal_id, data in anomalies.iterrows():
        
        print("\n" + "="*30)
        print(f"  SUSPICIOUS ACTOR ID: {principal_id}")
        print("="*30)
        
        print("  Behavioral Profile:")
        print(f"    - Total Events:     {int(data['event_count'])}")
        print(f"    - Error Count:      {int(data['error_count'])}")
        print(f"    - Unique Services:  {int(data['unique_services_used'])}")
        print(f"    - Unique Regions:   {int(data['unique_regions'])}")
        print(f"    - Unique IPs:       {int(data['unique_ips'])}")
        print(f"    - Night Events:     {int(data['night_events'])}")
        
        print(f"\n  Model Anomaly Score: {data['anomaly_score']:.4f}")
        print("="*30)

Starting  Reporting Anomalies...

--- 🚨 2 Anomalous Entities Detected! ---

  SUSPICIOUS ACTOR ID: AIDATFQR7NSC5AU2ZV3IE
  Behavioral Profile:
    - Total Events:     2642
    - Error Count:      239
    - Unique Services:  27
    - Unique Regions:   1
    - Unique IPs:       6
    - Night Events:     0

  Model Anomaly Score: -0.1676

  SUSPICIOUS ACTOR ID: AIDATFQR7NSC5U6Q3TMDR
  Behavioral Profile:
    - Total Events:     105
    - Error Count:      14
    - Unique Services:  6
    - Unique Regions:   1
    - Unique IPs:       4
    - Night Events:     0

  Model Anomaly Score: -0.0182
